In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from data import *
from window import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

start_date = (datetime.today() - timedelta(days=59)).strftime('%Y-%m-%d')
end_date = datetime.today().strftime('%Y-%m-%d')

ticker = 'SPY'
data = fetch_data(ticker, start_date, end_date)


def fetch_data(symbol, start_date, end_date):
    try:
        # Fetch historical data from Yahoo Finance
        stock_data = yf.download(symbol, start=start_date, end=end_date, interval='15m')
        stock_data.drop(columns=['Adj Close', 'Volume'], inplace=True)
        return stock_data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None
    

def information(prob):
     return -np.log(prob)


def calculate_frequency(data):
    # Calculate returns and bin them
    data['return'] = data['Close'].pct_change(periods=1)
    data['return_bin'] = pd.cut(data['return'], 10)
    
    # Calculate frequency of each bin
    freq = data['return_bin'].value_counts(normalize=True).reset_index()
    freq.columns = ['return_bin', 'count']
    
    # Initialize scaler
    scaler = StandardScaler()
    
    # Compute 'info' values and scale them
    info = information(freq['count']).values.reshape(-1,1)  # Convert Series to NumPy array
    freq['info'] = scaler.fit_transform(info)
    
    return freq


def get_information(price_change, freq):
    # Iterate over each row in the frequency DataFrame
    for index, row in freq.iterrows():
        # Check if the price change falls within the bin interval
        if row['return_bin'].left <= price_change < row['return_bin'].right:
            return row['info']
    
    # If no match is found, return None or handle the case accordingly
    return None

[*********************100%%**********************]  1 of 1 completed


In [3]:
freq = calculate_frequency(data)
price_change = 0.05  # Example of the most recent price change
information_value = get_information(price_change, freq)

print("Information value associated with the price change:", information_value)

Information value associated with the price change: None
